# Steane Code Logical T1 Calculation

In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt

from general_qec.qec_helpers import *
from general_qec.gates import *
from general_qec.errors import *
from circuit_specific.steane_logical_t1 import *

# using datetime module
import datetime # used to see how long things take here

## Contents
1. [Introduction](#introduction)
2. [Implementing the Steane code with relaxation and dephasing errors](#errors)
3. [Calculating the logical T1 for the Steane code bit correction circuit](#t1)
4. [More realistic error implementation](#realistic)

## 1. Introduction <a id='introduction'></a>
### In this file we will restrict our Steane code connectivity to a line connected circuit and implement our T1 and T2 error model to check for our logical T1 and T2 after running our 3 qubit correction circuit with line connectivity many times. We will also introduce depolarizing errors after we have implemented the T1 and T2 model.

# 2. Implementing the Steane code with Relaxation and dephasing errors  <a id='errors'></a>

#### We will now implement the relaxation and dephasing model created in 05. Error Models.

Since we implemented our Steane code using functions, we will need to pull those in here but change them up a bit.

In [ ]:
# Remember our stabilizer operators? 
# We will need to make a few changes to them and implement the CNOT gates one at a time on our density matrix 
# just like we did with our line connectivity operations

# Define the Stabilizer Operators as CNOT gates 
# (remember that the non-adj CNOT calculation is using line connectivity)
K1_line_operation = np.dot(CNOT(7, 3, 10), np.dot(CNOT(7, 4, 10), np.dot(
    CNOT(7, 5, 10), CNOT(7, 6, 10))))
K2_line_operation = np.dot(CNOT(8, 0, 10), np.dot(CNOT(8, 2, 10), np.dot(
    CNOT(8, 4, 10), CNOT(8, 6, 10))))
K3_line_operation = np.dot(CNOT(9, 1, 10), np.dot(CNOT(9, 2, 10), np.dot(
    CNOT(9, 5, 10), CNOT(9, 6, 10))))

K4_line_operation = np.dot(CZ(7, 3, 10), np.dot(CZ(7, 4, 10), np.dot(
    CZ(7, 5, 10), CZ(7, 6, 10))))
K5_line_operation =np.dot(CZ(8, 0, 10), np.dot(CZ(8, 2, 10), np.dot(
    CZ(8, 4, 10), CZ(8, 6, 10))))
K6_line_operation =np.dot(CZ(9, 1, 10), np.dot(CZ(9, 2, 10), np.dot(
    CZ(9, 5, 10), CZ(9, 6, 10))))

First we can initialize the state of our system and turn it into its density matrix form

In [ ]:
zero = np.array([1, 0])
one = np.array([0, 1])

# the parameters of our system
t1 = 200 * 10**-6 # 200 us - the initial T1
t2 = 150 * 10**-6 # 150 us - Make sure this is less than or equal to T1
tg = 20 * 10**-9 # 20 ns

# t1 = t2 = 1
# Set the initial states of your physical qubits
# initial_state = np.kron(zero, np.kron(zero, np.kron(zero, np.kron(zero, np.kron(zero, np.kron(zero, zero))))))
initial_state = np.kron(one, np.kron(one, np.kron(one, np.kron(one, np.kron(one, np.kron(one, one))))))
# initial_state = 1/np.sqrt(2) * np.kron(np.array([[1,1]]), np.kron(np.array([[1,1]]), np.kron(
#     np.array([[1,1]]), np.kron(np.array([[1,1]]), np.kron(np.array([[1,1]]), np.kron(
#         np.array([[1,1]]), np.array([[1,1]])))))))

# Set the initial state of your ancilla qubits
ancilla = np.kron(zero, np.kron(zero, zero))

# couple our ancilla qubits
initial_state = np.kron(initial_state, ancilla)

# output our initial state after coupling to the ancilla
print('initial state')
print_state_info(initial_state, 10)

# find the density matrix
initial_rho = np.kron(initial_state, initial_state[np.newaxis].conj().T)




First we will apply the Z correction protocol

In [ ]:
# probability of the state measurments from the density matrix are defined as Tr(p*rho)
state_probs = np.array([])
tot = 0
for i in range(len(initial_rho)):
    tot += np.abs(initial_rho[i, i])
    state_probs = np.append(state_probs, initial_rho[i,i])

initial_state = np.sqrt(state_probs)
print('Initial state before Z correction:')
print_state_info(initial_state, 10)
print(' - ')

### Implements the 7 Qubit Steane phase correction code using line connectivity

# - - - - - - - - - - # Z Error Correction # - - - - - - - - - - #
# apply the first hadamard to the ancillas
ancilla_hadamard = np.kron(np.identity(2**7), np.kron(hadamard, np.kron(hadamard, hadamard)))
current_rho = np.dot(ancilla_hadamard, np.dot(initial_rho, ancilla_hadamard.conj().T))
current_rho = qubit_rad_error_matrix(current_rho, t1, t2, tg)

# normalize rho after hadamard operation
current_rho = 8 * current_rho

# apply the control stabilizer gates to current_rho

# apply K1 first:
current_rho = line_rad_CNOT(current_rho, 7, 3, t1, t2, tg, form = 'rho')
current_rho = line_rad_CNOT(current_rho, 7, 4, t1, t2, tg, form = 'rho')
current_rho = line_rad_CNOT(current_rho, 7, 5, t1, t2, tg, form = 'rho')
current_rho = line_rad_CNOT(current_rho, 7, 6, t1, t2, tg, form = 'rho')

# apply K2:
current_rho = line_rad_CNOT(current_rho, 8, 0, t1, t2, tg, form = 'rho')
current_rho = line_rad_CNOT(current_rho, 8, 2, t1, t2, tg, form = 'rho')
current_rho = line_rad_CNOT(current_rho, 8, 4, t1, t2, tg, form = 'rho')
current_rho = line_rad_CNOT(current_rho, 8, 6, t1, t2, tg, form = 'rho')

# apply K3:
current_rho = line_rad_CNOT(current_rho, 9, 1, t1, t2, tg, form = 'rho')
current_rho = line_rad_CNOT(current_rho, 9, 2, t1, t2, tg, form = 'rho')
current_rho = line_rad_CNOT(current_rho, 9, 5, t1, t2, tg, form = 'rho')
current_rho = line_rad_CNOT(current_rho, 9, 6, t1, t2, tg, form = 'rho')

# apply the second hadamard to the ancillas
current_rho = np.dot(ancilla_hadamard, np.dot(current_rho, ancilla_hadamard.conj().T))
current_rho = qubit_rad_error_matrix(current_rho, t1, t2, tg)

# probability of the state measurments from the density matrix are defined as Tr(p*rho)
state_probs = np.array([])
tot = 0
for i in range(len(current_rho)):
    tot += np.abs(current_rho[i, i])
    state_probs = np.append(state_probs, current_rho[i,i])


# Measure the ancilla qubits and collapse them
collapsed_state = collapse_ancilla(np.sqrt(state_probs), 3)

print('Collapsed state after ancilla measurement:')
print_state_info(collapsed_state, 10)
print(' - ')

# Create our new density matrix after collapsing ancilla qubits
rho = np.kron(collapsed_state, collapsed_state[np.newaxis].conj().T)

# apply an error for time taken to collapse ancilla
rho = qubit_rad_error_matrix(rho, t1, t2, tg)

# How many total qubits are in our vector representation
n = int(np.log(len(collapsed_state))/np.log(2))

# Measure the three ancilla qubits
# Applying the Z gate operation on a specific qubit
bits = vector_state_to_bit_state(collapsed_state, n)[0][0]

# find index
m_one = 0
m_two = 0
m_three = 0
if bits[7] == '1':
    m_one = 1
if bits[8] == '1':
    m_two = 1
if bits[9] == '1':
    m_three = 1

# Which qubit do we perform the Z gate on
index = (m_one * 2**2) + (m_three * 2**1) + (m_two * 2**0) - 1

# if no error occurs we dont need to apply a correction
if index == -1:
    final_rho = rho

else:
    # apply the z gate depending on index
    operation = np.kron(np.identity(2**(index)), np.kron(sigma_z, np.kron(
        np.identity(2**(n-3-index-1)), np.identity(2**3))))
    
    final_rho = np.dot(operation, np.dot(rho, operation.conj().T))
    final_rho = qubit_rad_error_matrix(final_rho, t1, t2, tg) # apply an error for correction gate time

    
# probability of the state measurments from the density matrix are defined as Tr(p*rho)
state_probs = np.array([])
tot = 0
for i in range(len(final_rho)):
    tot += np.abs(final_rho[i,i])
    state_probs = np.append(state_probs, final_rho[i,i])

final_state_z = np.sqrt(state_probs)
print('Final state after Z correction:')
print_state_info(final_state_z, 10)


In [ ]:
# total gates applied when implementing the Z correction operators
total_gates_z = 1 + CNOT_gate_tot(7, 3) + CNOT_gate_tot(7, 4) + CNOT_gate_tot(7, 5) + 1 + CNOT_gate_tot(
    8, 0) + CNOT_gate_tot(8, 2) + CNOT_gate_tot(8, 4) + CNOT_gate_tot(8, 6) + CNOT_gate_tot(
    9, 1) + CNOT_gate_tot(9, 2) + CNOT_gate_tot(9, 5) + CNOT_gate_tot(9, 6) + 1 + 2

print('Number of gates applied: ', total_gates_z)
time_z = total_gates_z * tg
tot_time = time_z
print('Time in operation: ', time_z, 'sec')
print('Total time in circuit: ',tot_time , 'sec')

bits = vector_state_to_bit_state(final_state_z, 10)[0]

# Plotting the error state probabilities
fig, ax = plt.subplots()
fig.set_figwidth(20)
error_state_bars = ax.bar(bits, (final_state_z[final_state_z != 0])**2,
                          label = 'States Probability Distribution', color = 'cornflowerblue')
plt.title('Probability Distribution of States after Z correction')
plt.xlabel('logcial bit states') 
plt.ylabel('probability') 
ax.bar_label(error_state_bars)
plt.show()

Now lets apply the X error correction protocol

In [ ]:
# Reset the ancilla qubits:
initial_state = ancilla_reset(final_state_z, 3)
print('Initial state before X correction:')
print_state_info(initial_state, 10)
print(' - ')

### Implements the 7 Qubit Steane bit correction code using line connectivity
initial_rho = np.kron(initial_state, initial_state[np.newaxis].conj().T)
# initial_rho = np.kron(initial_state, initial_state[np.newaxis].conj().T)

# - - - - - - - - - - # X Error Correction # - - - - - - - - - - #

# apply the first hadamard to the ancillas
ancilla_hadamard = np.kron(np.identity(2**7), np.kron(hadamard, np.kron(hadamard, hadamard)))
current_rho = np.dot(ancilla_hadamard, np.dot(initial_rho, ancilla_hadamard.conj().T))
current_rho = qubit_rad_error_matrix(current_rho, t1, t2, tg)

# apply the control stabilizer gates to current_rho

# apply K4 first:
current_rho = line_rad_CZ(current_rho, 7, 3, t1, t2, tg, form = 'rho')
current_rho = line_rad_CZ(current_rho, 7, 4, t1, t2, tg, form = 'rho')
current_rho = line_rad_CZ(current_rho, 7, 5, t1, t2, tg, form = 'rho')
current_rho = line_rad_CZ(current_rho, 7, 6, t1, t2, tg, form = 'rho')

# apply K5:
current_rho = line_rad_CZ(current_rho, 8, 0, t1, t2, tg, form = 'rho')
current_rho = line_rad_CZ(current_rho, 8, 2, t1, t2, tg, form = 'rho')
current_rho = line_rad_CZ(current_rho, 8, 4, t1, t2, tg, form = 'rho')
current_rho = line_rad_CZ(current_rho, 8, 6, t1, t2, tg, form = 'rho')

# apply K6:
current_rho = line_rad_CZ(current_rho, 9, 1, t1, t2, tg, form = 'rho')
current_rho = line_rad_CZ(current_rho, 9, 2, t1, t2, tg, form = 'rho')
current_rho = line_rad_CZ(current_rho, 9, 5, t1, t2, tg, form = 'rho')
current_rho = line_rad_CZ(current_rho, 9, 6, t1, t2, tg, form = 'rho')

# apply the second hadamard to the ancillas
current_rho = np.dot(ancilla_hadamard, np.dot(current_rho, ancilla_hadamard.conj().T))
current_rho = qubit_rad_error_matrix(current_rho, t1, t2, tg)

# probability of the state measurments from the density matrix are defined as Tr(p*rho)
state_probs = np.array([])
tot = 0
for i in range(len(current_rho)):
    tot += np.abs(current_rho[i, i])
    state_probs = np.append(state_probs, current_rho[i,i])

# Measure the ancilla qubits and collapse them
collapsed_state = collapse_ancilla(np.sqrt(state_probs), 3)

print('Collapsed state after ancilla measurement:')
print_state_info(collapsed_state, 10)
print(' - ')

# Create our new density matrix after collapsing ancilla qubits
rho = np.kron(collapsed_state, collapsed_state[np.newaxis].conj().T)

# apply an error for time taken to collapse ancilla
rho = qubit_rad_error_matrix(rho, t1, t2, tg)

# How many total qubits are in our vector representation
n = int(np.log(len(collapsed_state))/np.log(2))

# Measure the three ancilla qubits
# Applying the X gate operation on a specific qubit
bits = vector_state_to_bit_state(collapsed_state, n)[0][0]

# find index
m_four = 0
m_five = 0
m_six = 0
if bits[7] == '1':
    m_four = 1
if bits[8] == '1':
    m_five = 1
if bits[9] == '1':
    m_six = 1

# Which qubit do we perform the Z gate on
index = (m_four * 2**2) + (m_six * 2**1) + (m_five * 2**0) - 1

# if no error occurs we dont need to apply a correction
if index == -1:
    final_rho = rho

else:
    # apply the z gate depending on index
    operation = np.kron(np.identity(2**(index)), np.kron(sigma_x, np.kron(
        np.identity(2**(n-3-index-1)), np.identity(2**3))))
    
    final_rho = np.dot(operation, np.dot(rho, operation.conj().T))
    final_rho = qubit_rad_error_matrix(final_rho, t1, t2, tg) # apply an error for correction gate time
    
    
# probability of the state measurments from the density matrix are defined as Tr(p*rho)
state_probs = np.array([])
tot = 0
for i in range(len(final_rho)):
    tot += np.abs(final_rho[i,i])
    state_probs = np.append(state_probs, final_rho[i,i])

final_state_x = np.sqrt(state_probs)
print('Final state after X correction:')
print_state_info(final_state_x, 10)

In [ ]:
# total gates applied when implementing X correction operators (for every CZ we add 2 for the hadamard converison)
total_gates_x = total_gates_z + (2*12) # CZ gates

print('Number of gates applied: ', total_gates_x)
time_x =  total_gates_x * tg
tot_time += time_x
print('Time in operation: ', time_x, 'sec')
print('Total time in circuit: ', tot_time, 'sec')
bits = vector_state_to_bit_state(final_state_x, 10)[0]

# Plotting the error state probabilities
fig, ax = plt.subplots()
fig.set_figwidth(20)
error_state_bars = ax.bar(bits, (final_state_x[final_state_x!=0])**2,
                          label = 'States Probability Distribution', color = 'cornflowerblue')
plt.title('Probability Distribution of States after X correction')
plt.xlabel('logcial bit states') 
plt.ylabel('probability') 
ax.bar_label(error_state_bars)
plt.show()

## 3. Calculating the logical T1 for the Steane code  <a id='t1'></a>

We will change parameters and loop over the number of iterations that we impelemnt the circuit

In [ ]:
zero = np.array([1, 0])
one = np.array([0, 1])

# the parameters of our system
t1 = 200 * 10**-6 # 200 us - the initial T1
t2 = 150 * 10**-6 # 150 us - Make sure this is less than or equal to T1
tg = 20 * 10**-9 # 20 ns
# t1 = t2 = 1
# Set the initial states of your physical qubits
# initial_state = np.kron(zero, np.kron(zero, np.kron(zero, np.kron(zero, np.kron(zero, np.kron(zero, zero))))))
initial_state = np.kron(one, np.kron(one, np.kron(one, np.kron(one, np.kron(one, np.kron(one, one))))))
# initial_state = np.kron(np.array([[1,1]]), np.kron(np.array([[1,1]]), np.kron(
#     np.array([[1,1]]), np.kron(np.array([[1,1]]), np.kron(np.array([[1,1]]), np.kron(
#         np.array([[1,1]]), np.array([[1,1]])))))))[0]

# Set the initial state of your ancilla qubits
ancilla = np.kron(zero, np.kron(zero, zero))

# couple our ancilla qubits
initial_state = np.kron(initial_state, ancilla)

# find the density matrix
initial_rho = np.kron(initial_state, initial_state[np.newaxis].conj().T)

# rho, vector = rad_steane(initial_rho, t1, t2, tg, info = True)

In [ ]:
all_probs = np.array([])
count = np.array([])
# add the state before we run the code
tot = 0
for i in range(len(initial_state)):
    count = np.append(count, 0)
    tot += np.abs(initial_rho[i,i])
    all_probs = np.append(all_probs, tot)

# Apply the function above 5 times to run the circuit
for j in range(5):
    count = np.append(count, j+1)
    if j == 0:
        rho, vector = realistic_steane(initial_rho, t1=t1, t2=t2, tg=tg, info = False)
        tot = 0
        for i in range(len(vector)):
            tot += np.abs(rho[i,i])
    else:
        rho, vector = realistic_steane(rho, t1=t1, t2=t2, tg=tg, info = False)
        tot = 0
        for i in range(len(vector)):
            tot += np.abs(rho[i,i])
    print('iteration: ', j+1)      
    all_probs = np.append(all_probs, tot)
    

In [ ]:
plt.scatter(count, all_probs, s = 1)
plt.title('12 Qubit population as a function of running Steane code')
plt.xlabel('number of iterations')
plt.ylabel('qubit population')
plt.axhline(y = 1/np.e, color = 'r', linestyle = '-')
# plt.axvline(x = 58, color = 'g', linestyle = '-')

plt.show()

Now we will loop over many times and see how our logical T1 changes based on physical T1 and T2 values

In [ ]:
# the parameters of our system
t1 = 100 * 10**-6 # 100 us - the initial T1
t2 = 100 * 10**-6 # 100 us - Make sure this is less than or equal to T1
tg = 20 * 10**-9 # 20 ns


# Set the initial states of your physical qubits
# initial_state = np.kron(zero, np.kron(zero, np.kron(zero, np.kron(zero, np.kron(zero, np.kron(zero, zero))))))
initial_state = np.kron(one, np.kron(one, np.kron(one, np.kron(one, np.kron(one, np.kron(one, one))))))
# initial_state = np.kron(np.array([[1,1]]), np.kron(np.array([[1,1]]), np.kron(
#     np.array([[1,1]]), np.kron(np.array([[1,1]]), np.kron(np.array([[1,1]]), np.kron(
#         np.array([[1,1]]), np.array([[1,1]])))))))[0]

# Set the initial state of your ancilla qubits
ancilla = np.kron(zero, np.kron(zero, zero))

# couple our ancilla qubits
initial_state = np.kron(initial_state, ancilla)

# find the density matrix
initial_rho = np.kron(initial_state, initial_state[np.newaxis].conj().T)

all_probs = np.array([[]])
all_counts = np.array([[]])
all_t1 = np.array([])
all_t2 = np.array([])
t1_start = t1
t2_start = t2

for k in range(10):
    all_t1 = np.append(all_t1, t1)
    all_t2 = np.append(all_t2, t2)
    
    probs = np.array([])
    count = np.array([])
    # Apply the function above 100 times to run the circuit 100 times
    for j in range(10):
        if j == 0:
            # add the state before we run the code
            tot = 0
            for i in range(len(initial_state)):
                count = np.append(count, 0)
                tot += np.abs(initial_rho[i,i])
                probs = np.append(probs, tot)
                
            count = np.append(count, j+1)
            rho, vector = realistic_steane(initial_rho, t1=t1, t2=t2, tg=tg, info = False)
            tot = 0
            for i in range(len(vector)):
                tot += np.abs(rho[i,i])
        else:
            count = np.append(count, j+1)
            rho, vector = realistic_steane(rho, t1=t1, t2=t2, tg=tg, info = False)
            tot = 0
            for i in range(len(vector)):
                tot += np.abs(rho[i,i])
        
#         print('iteration: ', j+1)      
        probs = np.append(probs, tot)
    
    if k == 0:
        all_probs = [probs]
        all_counts = [count]
    else:
        all_probs = np.append(all_probs, [probs], axis = 0)
        all_counts = np.append(all_counts, [count], axis = 0)
    
    t1 += 100 * 10 ** - 6 # add 100 us to T1
    t2 += 100 * 10** - 6 # add 100 us to T2

In [ ]:
# Creating 2-D grid of features
[X, Y] = np.meshgrid(all_counts[0], all_t1)

fig, ax = plt.subplots(1, 1)
fig.set_figwidth(8)
Z = all_probs
levels = np.arange(0, 1, 0.025)
levels = np.sort(np.append(levels, 1/np.e))

cp = plt.contourf(X, Y * 10**6, Z, levels)

cbar = plt.colorbar()
ax.set_title('Qubit population after iterations of Steane code')
ax.set_xlabel('iterations of Steane code')
ax.set_ylabel('physical t1/t2 (us)')
cbar.set_label('System qubit population')

print('Physical T1 range: (', t1_start,', ', t1, ') sec')
print('Physical T2 range: (',t2_start,', ', t2, ') sec')
print('Gate time (Tg): ', tg, 'sec')

# print('Depolarizing error by probability at each qubit: ', qubit_error_probs)
# show where the logical T1 is
plt.contour(cp, levels = sorted(levels[levels == 1/np.e]), colors=('r',),linestyles=('-',),linewidths=(2,))

plt.show()

In [ ]:
# Calculate the time spent in the circuit:
circuit_gates = total_gates_z + total_gates_x # find the number of gates after one iteration
tot_gates = circuit_gates * all_counts[0] # find the number of gates after each iteration
total_time = tot_gates * tg * 10**6 # convert to time in us

# Creating 2-D grid of features
[X, Y] = np.meshgrid(total_time, all_t1)

fig, ax = plt.subplots(1, 1)
fig.set_figwidth(8)
Z = all_probs
levels = np.arange(0, 1, 0.025)
levels = np.sort(np.append(levels, 1/np.e))

cp = plt.contourf(X, Y * 10**6, Z, levels)

cbar = plt.colorbar()
ax.set_title('Qubit population after time elapsed in Steane code')
ax.set_xlabel('Time elapsed (us)')
ax.set_ylabel('physical t1 and t2 (us)')
cbar.set_label('System qubit population')

print('Physical T1 range: (', t1_start,', ', t1, ') sec')
print('Physical T2 range: (',t2_start,', ', t2, ') sec')
print('Gate time (Tg): ', tg, 'sec')

# print('Depolarizing error by probability at each qubit: ', qubit_error_probs)
# show where the logical T1 is
plt.contour(cp, levels = sorted(levels[levels == 1/np.e]), colors=('r',),linestyles=('-',),linewidths=(2,))

plt.show()

## 4. More realistic error implementation <a id='realistic'></a>

Now we will not apply discrete errors as our error models will implement continuous errors on all gate operations and state preparation and measurement errors.

In [ ]:
zero = np.array([1, 0])
one = np.array([0, 1])

# the parameters of our system
t1 = 200 * 10**-6 # 200 us - the initial T1
t2 = 150 * 10**-6 # 150 us - Make sure this is less than or equal to T1
tg = 20 * 10**-9 # 20 ns

# state preparation and measurement errors
spam_prob = 0.00001

p_q0 = p_q1 = p_q2 = p_q3 = p_q4 = p_q5 = p_q6 = p_q7 = p_q8 = p_q9 = 1e-5
# define your error probability for each qubit
qubit_error_probs = np.array([p_q0, p_q1, p_q2, p_q3, p_q4, p_q5, p_q6, p_q7, p_q8, p_q9])

# Set the initial states of your physical qubits
# initial_state = np.kron(zero, np.kron(zero, np.kron(zero, np.kron(zero, np.kron(zero, np.kron(zero, zero))))))
initial_state = np.kron(one, np.kron(one, np.kron(one, np.kron(one, np.kron(one, np.kron(one, one))))))
# initial_state = np.kron(np.array([[1,1]]), np.kron(np.array([[1,1]]), np.kron(
#     np.array([[1,1]]), np.kron(np.array([[1,1]]), np.kron(np.array([[1,1]]), np.kron(
#         np.array([[1,1]]), np.array([[1,1]])))))))[0]

# Set the initial state of your ancilla qubits
ancilla = np.kron(zero, np.kron(zero, zero))

# couple our ancilla qubits
initial_state = np.kron(initial_state, ancilla)

# find the density matrix
initial_rho = np.kron(initial_state, initial_state[np.newaxis].conj().T)

rho, vector = realistic_steane(
    initial_rho, t1=t1, t2=t2, tg=tg, qubit_error_probs=qubit_error_probs, spam_prob=spam_prob, info = True)

In [ ]:
state_probs = np.array([])
tot = 0
for i in range(len(rho)):
    tot += np.abs(rho[i,i])
    state_probs = np.append(state_probs, rho[i,i])

vector = np.sqrt(state_probs)

In [ ]:
# Plot and output our information
print('Physical T1: ', t1, ' sec')
print('Physical T2: ', t2, ' sec')
print('Gate time (Tg): ', tg, 'sec')
print('Depolarizing error by probability at each qubit:\n', qubit_error_probs)

# total gates applied when implementing the Z correction operators
total_gates_z = 1 + CNOT_gate_tot(7, 3) + CNOT_gate_tot(7, 4) + CNOT_gate_tot(7, 5) + 1 + CNOT_gate_tot(
    8, 0) + CNOT_gate_tot(8, 2) + CNOT_gate_tot(8, 4) + CNOT_gate_tot(8, 6) + CNOT_gate_tot(
    9, 1) + CNOT_gate_tot(9, 2) + CNOT_gate_tot(9, 5) + CNOT_gate_tot(9, 6) + 1 + 2
total_gates_x = total_gates_z + 2*12
print('Number of gates applied: ', total_gates_z + total_gates_x)
tot_time = (total_gates_z + total_gates_x) * tg
print('Total time in circuit: ',tot_time , 'sec')

bits = vector_state_to_bit_state(vector, 10)[0]

# Plotting the error state probabilities
fig, ax = plt.subplots()
fig.set_figwidth(20)
error_state_bars = ax.bar(bits, (vector[vector != 0])**2,
                          label = 'States Probability Distribution', color = 'cornflowerblue')
plt.title('Probability Distribution of States after Steane Code')
plt.xlabel('logcial bit states') 
plt.ylabel('probability') 
ax.bar_label(error_state_bars)
plt.show()



In [ ]:
collapsed_state = collapse_ancilla(vector, 10)
print_state_info(collapsed_state, 7)

In [ ]:
# Plot and output our information with the main states
print('Physical T1: ', t1, ' sec')
print('Physical T2: ', t2, ' sec')
print('Gate time (Tg): ', tg, 'sec')
print('Depolarizing error by probability at each qubit:\n', qubit_error_probs)

# total gates applied when implementing the Z correction operators
total_gates_z = 1 + CNOT_gate_tot(7, 3) + CNOT_gate_tot(7, 4) + CNOT_gate_tot(7, 5) + 1 + CNOT_gate_tot(
    8, 0) + CNOT_gate_tot(8, 2) + CNOT_gate_tot(8, 4) + CNOT_gate_tot(8, 6) + CNOT_gate_tot(
    9, 1) + CNOT_gate_tot(9, 2) + CNOT_gate_tot(9, 5) + CNOT_gate_tot(9, 6) + 1 + 2
total_gates_x = total_gates_z + 2*12
print('Number of gates applied: ', total_gates_z + total_gates_x)
tot_time = (total_gates_z + total_gates_x) * tg
print('Total time in circuit: ',tot_time , 'sec')

# bits = vector_state_to_bit_state(vector, 10)[0]
larger_vector = vector
larger_vector[larger_vector < 0.001] = 0
larger_bits = vector_state_to_bit_state(larger_vector, 10)[0]
# Plotting the error state probabilities
fig, ax = plt.subplots()
fig.set_figwidth(20)
error_state_bars = ax.bar(larger_bits, (larger_vector[larger_vector != 0])**2,
                          label = 'States Probability Distribution', color = 'cornflowerblue')
plt.title('Probability Distribution of States after Steane Code')
plt.xlabel('logcial bit states') 
plt.ylabel('probability') 
ax.bar_label(error_state_bars)
plt.show()



Looping many times using the more realistic error model

In [ ]:
# the parameters of our system
t1 = 100 * 10**-6 # 100 us - the initial T1
t2 = 100 * 10**-6 # 100 us - Make sure this is less than or equal to T1
tg = 20 * 10**-9 # 20 ns

p_q0 = p_q1 = p_q2 = p_q3 = p_q4 = p_q5 = p_q6 = p_q7 = p_q8 = p_q9 = 1e-5
# define your error probability for each qubit
qubit_error_probs = np.array([p_q0, p_q1, p_q2, p_q3, p_q4, p_q5, p_q6, p_q7, p_q8, p_q9])

# state preparation and measurement errors
spam_prob = 0.00001

# Set the initial states of your physical qubits
# initial_state = np.kron(zero, np.kron(zero, np.kron(zero, np.kron(zero, np.kron(zero, np.kron(zero, zero))))))
initial_state = np.kron(one, np.kron(one, np.kron(one, np.kron(one, np.kron(one, np.kron(one, one))))))
# initial_state = np.kron(np.array([[1,1]]), np.kron(np.array([[1,1]]), np.kron(
#     np.array([[1,1]]), np.kron(np.array([[1,1]]), np.kron(np.array([[1,1]]), np.kron(
#         np.array([[1,1]]), np.array([[1,1]])))))))[0]

# Set the initial state of your ancilla qubits
ancilla = np.kron(zero, np.kron(zero, zero))

# couple our ancilla qubits
initial_state = np.kron(initial_state, ancilla)

# find the density matrix
initial_rho = np.kron(initial_state, initial_state[np.newaxis].conj().T)

all_probs = np.array([[]])
all_counts = np.array([[]])
all_t1 = np.array([])
all_t2 = np.array([])
t1_start = t1
t2_start = t2

for k in range(7):
    all_t1 = np.append(all_t1, t1)
    all_t2 = np.append(all_t2, t2)
    
    probs = np.array([])
    count = np.array([])
    # Apply the function above 100 times to run the circuit 100 times
    for j in range(15):
        if j == 0:
            # add the state before we run the code
            tot = 0
            for i in range(len(initial_state)):
                count = np.append(count, 0)
                tot += np.abs(initial_rho[i,i])
                probs = np.append(probs, tot)
                
            count = np.append(count, j+1)
            rho, vector = realistic_steane(
    initial_rho, t1=t1, t2=t2, tg=tg, qubit_error_probs=qubit_error_probs, spam_prob=spam_prob, info = False)
            tot = 0
            for i in range(len(vector)):
                tot += np.abs(rho[i,i])
        else:
            count = np.append(count, j+1)
            rho, vector = realistic_steane(
    rho, t1=t1, t2=t2, tg=tg, qubit_error_probs=qubit_error_probs, spam_prob=spam_prob, info = False)
            tot = 0
            for i in range(len(vector)):
                tot += np.abs(rho[i,i])
        
#         print('iteration: ', j+1)      
        probs = np.append(probs, tot)
    
    if k == 0:
        all_probs = [probs]
        all_counts = [count]
    else:
        all_probs = np.append(all_probs, [probs], axis = 0)
        all_counts = np.append(all_counts, [count], axis = 0)
    
    t1 += 150 * 10 ** - 6 # add 100 us to T1
    t2 += 150 * 10** - 6 # add 100 us to T2
    # ct stores current time
    ct = datetime.datetime.now()
    print("current time:-", ct)

In [ ]:
# Creating 2-D grid of features
[X, Y] = np.meshgrid(all_counts[0][all_counts[0] != 0], all_t1)

fig, ax = plt.subplots(1, 1)
fig.set_figwidth(8)
Z = all_probs
levels = np.arange(0, 1, 0.025)
levels = np.sort(np.append(levels, 1/np.e))

cp = plt.contourf(X, Y * 10**6, Z, levels)

cbar = plt.colorbar()
ax.set_title('Qubit population after iterations of Steane code')
ax.set_xlabel('iterations of Steane code')
ax.set_ylabel('physical t1/t2 (us)')
cbar.set_label('System qubit population')

print('Physical T1 range: (', t1_start,', ', t1, ') sec')
print('Physical T2 range: (',t2_start,', ', t2, ') sec')
print('Gate time (Tg): ', tg, 'sec')

# print('Depolarizing error by probability at each qubit: ', qubit_error_probs)
# show where the logical T1 is
plt.contour(cp, levels = sorted(levels[levels == 1/np.e]), colors=('r',),linestyles=('-',),linewidths=(2,))

plt.show()